# LangChain: Models, Prompts and Output Parsers

## Outline

 * Direct API calls to GroqAPI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [5]:
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def get_completion(prompt, model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1 # 1 seems to give the best results
    )
    return response.choices[0].message.content


#### The code below used to select the model version at the time of code execution. Nonetheless, because I am using the GroqAPI, I have much more flexibility in choosing the LLM to promt, and the one used in these notebooks is Meta's LLaMA3 70b model with a context window of 8,192 tokens.

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : GroqAI

Let's start with a direct API calls to Groq.

In [3]:
get_completion("What is 1+1?", model="llama3-70b-8192")


'The answer to 1+1 is 2.'

In [8]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [9]:
style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [10]:
response = get_completion(prompt, model="llama3-70b-8192")
print(response)

Here is the translated text:

```
I'm extremely frustrated because my blender lid suddenly came loose and splattered my kitchen walls with smoothie. And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, please.
```

Note: I've transformed the pirate-themed language into a calm and respectful tone, using American English. I've also removed the colloquialisms and slang to make the language more polite and professional.


In [12]:
response = get_completion("How can I use Langchain to prompt the GroqAPI?", model="mixtral-8x7b-32768")
print(response) ## Just playing around, but the llm hulucinations are quite interesting!

I'm sorry for any confusion, but it seems there may be a misunderstanding. Langchain is a hypothetical concept that combines the functionalities of a programming language and a blockchain, and as such, it doesn't exist and cannot be used to prompt the GroqAPI or any other API.

GroqAPI, on the other hand, is an API provided by Groq for interacting with their data processing and querying system. To use the GroqAPI, you would typically need to use a programming language that can send HTTP requests, such as Python, JavaScript, or Go.

Here is an example of how you might use Python to send a query to the GroqAPI:

```python
import requests

# Set up the API endpoint and headers
url = "https://api.groq.com/example_project/query"
headers = {"Content-Type": "application/vnd.api+json"}

# Set up the query and data
query = 'grq("select * from example where value > 42")'
data = {'query': query}

# Send the query to the API and print the response
response = requests.post(url, headers=headers, jso

## Chat API : LangChain

Let's try how we can do the same using LangChain. To use GroqAPi, we must install 'langchain-groq' instead of 'langchain', see [Official Docs](https://python.langchain.com/v0.2/docs/integrations/chat/groq/) for refrence

In [14]:
#!pip install --upgrade langchain-groq

Note: you may need to restart the kernel to use updated packages.


### Model

In [4]:
from langchain_groq import ChatGroq #Langchai abstraction for GroqAPI endpoint
from langchain_core.prompts import ChatPromptTemplate

chat = ChatGroq(temperature=0.0, model_name="llama3-70b-8192")

system = "You are a helpful assistant."
human = "{text}"

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
print(chain.invoke({"text": "Explain the importance of low latency LLMs."}).content)

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Conversational AI**: In conversational AI, low-latency LLMs enable more natural and human-like interactions. Fast response times are critical in chatbots, virtual assistants, and voice assistants to maintain user engagement and provide a seamless experience.
2. **Real-time Language Translation**: Low-latency LLMs are vital in real-time language translation applications, such as simultaneous interpretation, live subtitles, and multilingual customer support. Fast processing ensures that translations are provided quickly, enabling effective communication across languages.
3. **Sentiment Analysis and Feedback**: In applications like customer service, social media monitoring, and review analysis, low-latency LLMs help analyze user feedback and sentiment in real-time. This enables businesses to 

In [5]:
chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x769860111780>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x769860113100>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

### Prompt template

In [6]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [7]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [8]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [9]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [10]:
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [11]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages).content
print(customer_response)

Here is the translated text in American English, in a calm and respectful tone:

"I'm extremely frustrated that my blender lid came loose and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now, please."


In [13]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

service_messages[0].content

"Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"

In [14]:
service_response = chat.invoke(service_messages).content
print(service_response)

```Ahoy, valued customer! 

I be regretful to inform ye that yer warranty don't cover the cleanin' expenses for yer kitchen, matey. It seems ye forgot to secure the lid o' yer blender afore startin' it up, and that be a mighty big no-no! Alas, that be yer responsibility, not ours. 

So, I be wishin' ye the best o' luck in gettin' yer kitchen shipshape again, but we can't be helpin' ye out with the costs, savvy? May the winds o' fortune blow in yer favor, and may ye be more mindful o' yer blender's needs in the future! Fair winds, me hearty!```


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [15]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [16]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

system_message= """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [17]:
prompt_template = ChatPromptTemplate.from_template(system_message)

In [19]:
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [20]:
messages = prompt_template.format_messages(text=customer_review)

chain = prompt | chat
print(chain.invoke(messages).content)

Here is the extracted information in JSON format:

```
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

Let me know if you need any further assistance!


In [22]:
type(chain.invoke(messages).content)

str

### Parse the LLM output string into a Python dictionary

In [23]:
from langchain.output_parsers import  ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [24]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [25]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [27]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [30]:
parsed_output = output_parser.parse(chain.invoke(messages).content)
parsed_output

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [31]:
type(parsed_output)

dict

In [32]:
parsed_output.get('delivery_days')

2